You can explore other ploting libraries like plotly, altair and highcharts

In [1]:
import pickle
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go
import sys
sys.path.append('../')
from modules.json_reader import *

with open('../components/review_embedding.pkl', 'rb') as handle:
    x = pickle.load(handle)
df = pd.DataFrame(x)

# make sure the data is stored in raw_data
metadata = metadata_loader("../raw_data/meta_Video_Games.json.gz").\
           dropna().\
           reset_index(drop=True)

df_merged = df.merge(metadata,how='inner',on='asin')

Class Imbalance is very dangerous and we need address it accordingly

In [16]:
labels = list(df.overall.value_counts().to_dict().keys())
data = df.overall.value_counts().to_list()

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=data, hole=.3)])

fig.update_layout(
    title_text="Amazon Product Review Rating Aggregated Count",
    autosize=False,
    minreducedwidth=250,
    minreducedheight=250,
    width=450,
    height=450
)
fig.show()

Length of each product review might be interesting to know as well 

In [127]:
import plotly.express as px

df_merged['length'] = df_merged['reviewText'].apply(lambda x: len(x))
length_analysis = df_merged.groupby(['overall','categories'],as_index=False)['length'].mean()

fig = px.bar(length_analysis, y='length', x='overall', text_auto='.2s',color='categories',
            title="Default: various text sizes, positions and angles")

fig.update_layout(
    title_text="Average review length per rating class and product category",
    autosize=False,
    minreducedwidth=250,
    minreducedheight=250,
    width=1850,
    height=800
)
fig.show()

I see that you have a density plot for price, but you can maybe try adding dimensionality to plots with aggregations. Most of the time, a simple density plot will not bring a lot of data insights to stakeholders. 
More dimension and break downs will convince your target audience 

Interesting enough, expensive products are rated higher ! Product Quality?!

In [91]:
import plotly.express as px

# values = []
# for i in [1,2]:
#     values.append(df_merged.loc[df_merged.overall==i].price.to_numpy())

group_labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5']
# lets sample. Too naive to plot raw data points - its too large
fig = px.box(df_merged, x="overall", y="price", points="all")
fig.update_layout(
    title_text="Boxplot and Data Distribution of each Rating Class",
    autosize=True,
    width=800,
    height=600
)
fig


I think it will be interesting to also show how the products are embedded in a 2D space. This will give us a better understanding of the performance of the sentence transformer

In [35]:
metadata = metadata_loader("../raw_data/meta_Video_Games.json.gz").\
           dropna().\
           reset_index(drop=True)

In [58]:
with open('../components/product_embedding.pkl', 'rb') as handle:
    x = pickle.load(handle)
products = pd.DataFrame(x)

ls = []
for i in range(len(products)):
    ls.append(products['embedding'][i].tolist())
array_data = np.array(ls)


In [66]:
from sklearn.decomposition import PCA
import plotly.express as px

pca = PCA(n_components=2)
result = pca.fit_transform(array_data)
result = pd.DataFrame(result,columns=['pca1','pca2'])
result = pd.concat([result,products.asin],axis=1).merge(metadata[['title','asin','price','description']],how='left',on='asin').dropna()

fig = px.scatter(result , x="pca1", y="pca2",color="title")
fig.show()


In [132]:
# 3D 
pca = PCA(n_components=3)
result = pca.fit_transform(array_data)
result = pd.DataFrame(result,columns=['pca1','pca2','pca3'])
result = pd.concat([result,products.asin],axis=1).merge(metadata[['title','asin','price','description']],how='left',on='asin').dropna()

fig = px.scatter_3d(result, x='pca1', y='pca2', z='pca3',color="title")
fig.show()